# Xarray Interpolation, Groupby, Resample, Rolling, and Coarsen

**Attribution**: This notebook is a revision of the [Xarray Interpolation, Groupby, Resample, Rolling, and Coarsen notebook](https://earth-env-data-science.github.io/lectures/xarray/xarray-part2.html) by Ryan Abernathey from [An Introduction to Earth and Environmental Data Science](https://earth-env-data-science.github.io/intro.html). Thanks to Aiyin Zhang for preparing this notebook. 

In this lesson, we cover some more advanced aspects of `xarray`.

You can access this notebook (in a Docker image) on this [GitHub repo](https://github.com/HamedAlemo/xarray-tutorial).

In [ ]:
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt

## Interpolation

In the previous lesson on `xarray`, we learned how to select data based on its dimension coordinates and align data with dimension different coordinates.
But what if we want to estimate the value of the data variables at _different coordinates_.
This is where interpolation comes in.

In [ ]:
# we write it out explicitly so we can see each point.
x_data = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
f = xr.DataArray(x_data**2, dims=['x'], coords={'x': x_data})
f

In [ ]:
f.plot(marker='o')

In [ ]:
f.sel(x=3)

We only have data on the integer points in x.
But what if we wanted to estimate the value at, say, 4.5?

In [ ]:
f.sel(x=4.5)

Interpolation to the rescue!

In [ ]:
f.interp(x=4.5)

Interpolation uses [scipy.interpolate](https://docs.scipy.org/doc/scipy/reference/interpolate.html) under the hood.
There are different modes of interpolation.

In [ ]:
f.interp(x=4.5, method='linear').values

In [ ]:
f.interp(x=4.5, method='nearest').values

In [ ]:
f.interp(x=4.5, method='cubic').values

We can interpolate to a whole new set of coordinates at once:

In [ ]:
x_new = x_data + 0.5
f_interp_linear = f.interp(x=x_new, method='linear')
f_interp_cubic = f.interp(x=x_new, method='cubic')
f.plot(marker='o', label='original')
f_interp_linear.plot(marker='o', label='linear')
f_interp_cubic.plot(marker='o', label='cubic')
plt.legend()

Note that values outside of the original range are not supported:

In [ ]:
f_interp_cubic.values

```{note}
You can apply interpolation to any dimension, and even to multiple dimensions at a time.
(Multidimensional interpolation only supports `mode='nearest'` and `mode='linear'`.)
But keep in mind that `xarray` has no built-in understanding of geography.
If you use `interp` on lat / lon coordinates, it will just perform naive interpolation of the lat / lon values.
More sophisticated treatment of spherical geometry requires another package such as [xesmf](https://xesmf.readthedocs.io/).
```

## Loading Data from NetCDF Files

NetCDF (Network Common Data Format) is the most widely used format for distributing geoscience data. NetCDF is maintained by the [Unidata](https://www.unidata.ucar.edu/) organization.

Below we quote from the [NetCDF website](https://www.unidata.ucar.edu/software/netcdf/docs/faq.html#whatisit):

>NetCDF (network Common Data Form) is a set of interfaces for array-oriented data access and a freely distributed collection of data access libraries for C, Fortran, C++, Java, and other languages. The netCDF libraries support a machine-independent format for representing scientific data. Together, the interfaces, libraries, and format support the creation, access, and sharing of scientific data.
>
>NetCDF data is:
>
> - Self-Describing. A netCDF file includes information about the data it contains.
> - Portable. A netCDF file can be accessed by computers with different ways of storing integers, characters, and floating-point numbers.
> - Scalable. A small subset of a large dataset may be accessed efficiently.
> - Appendable. Data may be appended to a properly structured netCDF file without copying the dataset or redefining its structure.
> - Sharable. One writer and multiple readers may simultaneously access the same netCDF file.
> - Archivable. Access to all earlier forms of netCDF data will be supported by current and future versions of the software.

`xarray` is designed to make reading netCDF files in python as easy, powerful, and flexible as possible. (See [xarray netCDF docs](http://xarray.pydata.org/en/latest/io.html#netcdf) for more details.)

Below we download and load some the NASA [GISSTemp](https://data.giss.nasa.gov/gistemp/) global temperature anomaly dataset. The original file is located at https://data.giss.nasa.gov/pub/gistemp/gistemp1200_GHCNv4_ERSSTv5.nc.gz, but their server is non-responsive at the time of publication of this notebook. So we will use the mirror server from Zenodo. 

In [ ]:
import pooch

In [ ]:
gistemp_file = pooch.retrieve(
    'https://zenodo.org/records/13963679/files/gistemp1200_GHCNv4_ERSSTv5.nc',
    known_hash=None
)

In [ ]:
ds = xr.open_dataset(gistemp_file)
ds

In [ ]:
ds.tempanomaly.isel(time=-1).plot()
plt.show()

In [ ]:
ds.tempanomaly.mean(dim=('lon', 'lat')).plot()
plt.show()

Let's visualize the temperature anomally for the grid near Worcester. (Worcester's coordinates: latitude: 42.2626; longtitude: -71.8023)

In [ ]:
ds.tempanomaly.sel(lat=42.2626, lon=-71.8023, method='nearest').plot()
plt.show()

Note that you need to define `method` since the corresponding values for Worcester latitude and longitude do not exist in the `DataArray`'s coordinates. 

## Groupby

`xarray` copies `pandas'` very useful groupby functionality, enabling the "split / apply / combine" workflow on `xarray` `DataArray`s and `Dataset`s. In the first part of the lesson, we will learn to use groupby by analyzing sea-surface temperature data.


First we load a dataset. We will use the [NOAA Extended Reconstructed Sea Surface Temperature (ERSST) v5](https://www.ncdc.noaa.gov/data-access/marineocean-data/extended-reconstructed-sea-surface-temperature-ersst-v5) product, a widely used and trusted gridded compilation of historical data going back to 1854.

Since the data is provided via an [OPeNDAP](https://en.wikipedia.org/wiki/OPeNDAP) server, we can load it directly without downloading anything:

In [ ]:
url = 'http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/noaa.ersst.v5/sst.mnmean.nc'
ds = xr.open_dataset(url, drop_variables=['time_bnds'])

In [ ]:
ds = ds.sel(time=slice('1960', '2022'))
ds

Let's do some basic visualizations of the data, just to make sure it looks reasonable.

In [ ]:
ds.sst.sel(time = '1960-01-01').plot(ax = ax, vmin=-2, vmax=30)
plt.show()

Note that xarray correctly parsed the time index, resulting in a Pandas datetime index on the time dimension.

In [ ]:
ds.time

In [ ]:
ds.sst.sel(lon=300, lat=50).plot()
plt.show()

As we can see from the plot, the timeseries at any one point is totally dominated by the seasonal cycle. We would like to remove this seasonal cycle (called the "climatology") in order to better see the long-term variaitions in temperature. We will accomplish this using **groupby**.

The syntax of `xarray`'s groupby is almost identical to `pandas`.
We will first apply groupby to a single `DataArray`.

### Split Step

The most important argument is `group`: this defines the unique values we will use to "split" the data for grouped analysis. We can pass either a `DataArray` or a name of a variable in the dataset. Lets first use a `DataArray`. Just like with `pandas`, we can use the time index to extract specific components of dates and times. `xarray` uses a special syntax for this `.dt`, called the `DatetimeAccessor`.

See a list of datatime properties you can access through .dt [here](https://pandas.pydata.org/docs/user_guide/timeseries.html#time-date-components)

In [ ]:
ds.time.dt

In [ ]:
ds.time.dt.month

In [ ]:
ds.time.dt.year

We can use these arrays in a groupby operation:

In [ ]:
gb = ds.sst.groupby(ds.time.dt.month)
gb

`xarray` also offers a more concise syntax when the variable you're grouping on is already present in the dataset. This is identical to the previous line:

In [ ]:
gb = ds.sst.groupby('time.month')
gb

Now that the data are split, we can manually iterate over the group. The iterator returns the key (group name) and the value (the actual dataset corresponding to that group) for each group.

In [ ]:
for group_name, group_da in gb:
    print(group_name) # stop after first iteration
    break
group_da

### Map & Combine

Now that we have groups defined, it's time to "apply" a calculation to the group. Like in `pandas`, these calculations can either be:
- _aggregation_: reduces the size of the group
- _transformation_: preserves the group's full size

At then end of the apply step, `xarray` will automatically combine the aggregated / transformed groups back into a single object.

```{warning}
Xarray calls the "apply" step `map`. This is different from Pandas!
```

In [ ]:
gb.map?

#### Aggregations

Like `pandas`, `xarray`'s groupby object has many built-in aggregation operations (e.g. `mean`, `min`, `max`, `std`, etc):

In [ ]:
sst_mm = gb.mean(dim='time')
sst_mm

`.map` accepts as its argument a function. We can pass an existing function:

In [ ]:
gb.map(np.mean)

Because we specified no extra arguments (like `axis`) the function was applied over all space and time dimensions. This is not what we wanted. Instead, we could define a custom function. This function takes a single argument --the group dataset-- and returns a new dataset to be combined:

In [ ]:
def time_mean(a):
    return a.mean(dim='time')

sst_mm = gb.map(time_mean)
sst_mm

So we did what we wanted to do: calculate the climatology at every point in the dataset. Let's look at the data a bit.

_Climatlogy at a specific point in the North Atlantic_

In [ ]:
sst_mm.sel(lon=300, lat=-50).plot()
plt.show()

_Zonal Mean Climatology_

In [ ]:
sst_mm.mean(dim='lon').transpose().plot.contourf(levels=12, vmin=-2, vmax=30)
plt.show()

_Difference between January and July Climatology_

In [ ]:
(sst_mm.sel(month=1) - sst_mm.sel(month=7)).plot(vmax=10)
plt.show()

#### Transformations

Now we want to _remove_ this climatology from the dataset, to examine the residual, called the _anomaly_, which is the interesting part from a climate perspective.
Removing the seasonal climatology is a perfect example of a transformation: it operates over a group, but doesn't change the size of the dataset. Here is one way to code it.

In [ ]:
def remove_time_mean(x):
    return x - x.mean(dim='time')

ds_anom = ds.groupby('time.month').map(remove_time_mean)
ds_anom

```{note}
In the above example, we applied `groupby` to a `Dataset` instead of a `DataArray`.
```

Xarray makes these sorts of transformations easy by supporting _groupby arithmetic_.
This concept is easiest explained with an example:

In [ ]:
gb = ds.groupby('time.month')
ds_anom = gb - gb.mean(dim='time')
ds_anom

Now we can view the climate signal without the overwhelming influence of the seasonal cycle.

_Timeseries at a single point in the North Atlantic_

In [ ]:
ds_anom.sst.sel(lon=300, lat=50).plot()
plt.show()

_Difference between Jan. 1 2022 and Jan. 1 1960_

In [ ]:
(ds_anom.sel(time='2022-01-01') - ds_anom.sel(time='1960-01-01')).sst.plot()
plt.show()

## Grouby-Related: Resample, Rolling, Coarsen

### Resample

[Resample](https://docs.xarray.dev/en/v2023.10.1/generated/xarray.Dataset.resample.html) in `xarray` is nearly identical to `pandas`.
**It can be applied only to time-index dimensions.** Here we compute the five-year mean.
It is effectively a group-by operation, and uses the same basic syntax.
Note that resampling changes the length of the the output array.

See the [list](https://pandas.pydata.org/docs/user_guide/timeseries.html#offset-aliases) of string aliases for time series frequencies 

In [ ]:
ds_anom_resample = ds_anom.resample(time='5YE').mean(dim='time')
ds_anom_resample

In [ ]:
ds_anom.sst.sel(lon=300, lat=50).plot()
ds_anom_resample.sst.sel(lon=300, lat=50).plot(marker='o')
plt.show()

In [ ]:
(ds_anom_resample.sel(time='2015-01-01', method='nearest') -
 ds_anom_resample.sel(time='1965-01-01', method='nearest')).sst.plot()
plt.show()

### Rolling

[Rolling](https://docs.xarray.dev/en/v2023.10.1/generated/xarray.Dataset.rolling.html#xarray.Dataset.rolling) is also similar to `pandas`.
It does not change the length of arrays.
Instead, it allows a moving window to be applied to the data at each point.

In [ ]:
ds_anom_rolling = ds_anom.rolling(time=12, center=True).mean()
ds_anom_rolling

In [ ]:
ds_anom.sst.sel(lon=300, lat=50).plot(label='monthly anom')
ds_anom_resample.sst.sel(lon=300, lat=50).plot(marker='o', label='5 year resample')
ds_anom_rolling.sst.sel(lon=300, lat=50).plot(label='12 month rolling mean', color='k')
plt.legend()
plt.show()

## Coarsen

Coarsen is a simple way to reduce the size of your data along one or more axes.
It is very similar to `resample` when operating on time dimensions; the key difference is that coarsen only operates on fixed blocks of data, irrespective of the coordinate values, while resample actually looks at the coordinates to figure out, e.g. what month a particular data point is in. 

For regularly-spaced monthly data beginning in January, the following should be equivalent to annual resampling.
However, results would be different for irregularly-spaced data.

In [ ]:
ds.coarsen(time=12, boundary = 'exact').mean()

Coarsen also works on spatial coordinates (or any coordiantes).

In [ ]:
ds_coarse = ds.coarsen(lon=4, lat=4, boundary='pad').mean()
ds_coarse.sst.isel(time=0).plot(vmin=2, vmax=30, figsize=(12, 5), edgecolor='k')
plt.show()